In [1]:
import pandas as pd
import numpy as np
import streamlit as st
import openai
import time
from bs4 import BeautifulSoup
import requests
import re
import os
import json

from openai._client import OpenAI
from openai import AzureOpenAI


client = AzureOpenAI(
            azure_endpoint="https://da-openai-test.openai.azure.com/",
            api_key="5a90b85ba420469c9b36438e238d70fe",
            api_version="2023-05-15",
        )


In [2]:
today = time.strftime("%m_%d")
today_hour = time.strftime("%m_%d_%H")

# Make a folder for today's data inside the data folder
if not os.path.exists("data/" + today):
    os.mkdir("data/" + today)


In [4]:

# file_path_original_dataframe_csv = "data/all_products_block123.csv"
file_path_original_dataframe_excel = "data/12_15/All Products.xlsx"


new_file_path_excel = "data/"+today+"/software_tags" + today_hour + ".xlsx"
new_file_csv = "data/"+today+"/software_tags" + today_hour + ".csv"


In [5]:
df = pd.read_excel(file_path_original_dataframe_excel)
df.to_csv(new_file_csv, index=False)


In [21]:
# Create a new column to store the tags for each product
# df_tech_new_tags = pd.read_csv("./data/tags_30_11.csv", usecols=["Product_Name", "Product category", "url", "image_url"])
df_tech_new_tags = pd.read_csv(new_file_csv)

# Iterate over the rows of the dataframe
for index, row in df_tech_new_tags.iterrows():
    if pd.isna(row["is_software"]):
        print(index)
        product_category = str(row["Product category"])  # Convert to string
        # Get the tags for the product category
        print(row["Product_Name"])
        print(row["Product category"])
        if not pd.isna(row["Data sheets"]):
            print(row["Data sheets"])
        intro_text = str(row["Introduction"]) if pd.notna(row["Introduction"]) else ''
        block1 = str(row["Block1"]) if pd.notna(row["Block1"]) else ''
        block2 = str(row["Block2"]) if pd.notna(row["Block2"]) else ''
        block3 = str(row["Block3"]) if pd.notna(row["Block3"]) else ''
        Features = str(row["Features"]) if pd.notna(row["Features"]) else ''
        Technical_specifications = str(row["Technical Specifications"]) if pd.notna(row["Technical Specifications"]) else ''

        # Concatenate all the text
        website_text = intro_text + block1 + block2 + block3 + Features + Technical_specifications
        messages = [
            {
                "role": "system",
                "content": "You are a helpful assistant, return True if the product is a software product and False if it is a hardware product."
            },
            {
                "role": "user",
                "content": f"""For the product named '{row['Product_Name']}' with the following description: '{website_text}', is this a software product or a hardware product? Return True if it is a software product and False if it is a hardware product. Always return True with capital T and False with capital F."""
            }
        ]

        response = client.chat.completions.create(
            model="gpt-4-turbo-1106-preview",
            messages=messages,
            timeout=60,

        )

        tags = response.choices[0].message.content.strip()
        df_tech_new_tags.loc[index, "is_software"] = tags

    
        print("______________")
        print(tags)
        print("______________")
        df_tech_new_tags.loc[index, "product_tags"] = tags
        df_tech_new_tags.to_csv(new_file_csv, index=False)
        df_tech_new_tags.to_excel(new_file_path_excel, index=False)

# save the dataframe as csv
df_tech_new_tags.to_csv(new_file_csv, index=False)
df_tech_new_tags.to_excel(new_file_path_excel, index=False)

In [22]:
df_tech_new_tags.head(10)

,Unnamed: 0,Product_Name,Product category,Introduction,Block1,Block2,Block3,Features,Technical Specifications,url,Data sheets,downloads,product_family_name,is_range,Hero Image,Other Image,is_software,product_tags
0,0,"Seapath 380 - utilising GPS, GLONASS, Galileo...","Seafloor mapping, Inertial solutions","{'heading': 'Introduction', 'text': 'The Seapa...","{'heading': 'Advanced Navigation Integration',...",{'heading': 'Functional Design and System Conf...,"{'heading': 'Product Range and Applications', ...",• 0.007° to 0.02° roll and pitch accuracy depe...,PERFORMANCE POWER SPECIFICATIONS\nHeave accura...,NaN,https://www.kongsberg.com/contentassets/c22a59...,{'Data sheet': ['https://www.kongsberg.com/con...,NaN,NaN,NaN,NaN,False,False
1,1,12 kHz dual-beam transducer,"Seafloor mapping, Geophysical survey","{'heading': 'Introduction', 'text': 'The 12-16...","{'heading': 'Dual-Beam Functionality', 'text':...","{'heading': 'Beam Characteristics', 'text': 'T...",{'heading': 'Technical and Acoustic Performanc...,Resonant frequency: 12 kHz\nDual beam transduc...,Resonant frequency: 12 kHz\nMaximum transducer...,https://www.kongsberg.com/maritime/products/oc...,https://www.kongsberg.com/contentassets/d106c9...,NaN,Single beam bottom mapping transducers,1.0,['https://www.kongsberg.com/contentassets/e811...,NaN,False,False
2,2,120 kHz single-beam transducer,"Seafloor mapping, Geophysical survey","{'heading': 'Introduction', 'text': 'The 120-2...",{'heading': 'Order Number and Technical Overvi...,{'heading': 'Performance and Construction Deta...,{'heading': 'Acoustic Qualities and Efficiency...,"Resilient Design: Housed in a robust casing, t...",Resonant frequency: 120 kHz Maximum input powe...,https://www.kongsberg.com/maritime/products/oc...,https://www.kongsberg.com/contentassets/d106c9...,NaN,Single beam bottom mapping transducers,1.0,['https://www.kongsberg.com/contentassets/e811...,NaN,False,False
3,3,18 kHz single-beam transducer,"Seafloor mapping, Geophysical survey","{'heading': 'Introduction', 'text': 'The 18-11...","{'heading': 'Key Specifications', 'text': 'The...",{'heading': 'Physical Attributes and Operating...,"{'heading': 'Manufacturing Details', 'text': '...",Key Features:\n- Resonant frequency: 18 kHz\n-...,Technical Specifications:\n- Resonant frequenc...,https://www.kongsberg.com/maritime/products/oc...,https://www.kongsberg.com/contentassets/d106c9...,NaN,Single beam bottom mapping transducers,1.0,['https://www.kongsberg.com/contentassets/e811...,NaN,False,False
4,4,200 kHz single-beam transducer in robust housing,"Seafloor mapping, Geophysical survey","{'heading': 'Introduction', 'text': 'The 200-2...",{'heading': 'Robust Design & Deepwater Capabil...,{'heading': 'High Performance and Efficient Op...,{'heading': 'Durable Construction and Environm...,200 kHz single-beam transducer\nRobust housing...,"Resonant frequency, 200 kHz\nElectroacoustic e...",https://www.kongsberg.com/maritime/products/oc...,https://www.kongsberg.com/contentassets/d106c9...,NaN,Single beam bottom mapping transducers,1.0,['https://www.kongsberg.com/contentassets/e811...,NaN,False,False
5,5,3710 DGNSS RECEIVER,Navigation & positioning,"{'heading': 'Introduction', 'text': 'The 3710 ...",{'heading': 'Enhanced availability DGNSS proce...,"{'heading': 'Multiplexing Capability', 'text':...",{'heading': 'Receiver and Antenna Information'...,• High-performance DGNSS receiver\n• Capable o...,WEIGHTS AND DIMENSIONS\n- Receiver Unit: 44 x ...,https://www.kongsberg.com/maritime/products/ve...,https://www.kongsberg.com/contentassets/716271...,NaN,GNSS-based Relative Position Sensor,1.0,['https://www.kongsberg.com/contentassets/f3d2...,NaN,False,False
6,6,38 kHz Single-beam transducer,"Seafloor mapping, Geophysical survey","{'heading': 'Introduction', 'text': 'The 38-7 ...","{'heading': 'Transducer Overview', 'text': 'Th...","{'heading': 'Performance Characteristics', 'te...","{'heading': 'Operational Parameters', 'text': ...",- Incorporates 88 Tonpilz ele